In [32]:
import requests
import json 
from tqdm import tqdm

token="your_token"

In [33]:
def relevant_citations(impact_factor_year, bibcode):
    payload = {"bibcodes": [bibcode],
               "types": ["histograms"],
              "histograms": ["citations"]}
    response = requests.post("https://api.adsabs.harvard.edu/v1/metrics", \
                           headers={'Authorization': 'Bearer ' + token, 
                                    "Content-type": "application/json"}, \
                           data=json.dumps(payload))
    if response.status_code == 200:
        try:
            temp = response.json()
            relevant_year = str(impact_factor_year - 1)
            citations_histogram = temp.get('histograms', {}).get('citations', {})

            if citations_histogram:
                refereed_to_refereed = citations_histogram.get('refereed to refereed', {}).get(relevant_year, 0)
                refereed_to_nonrefereed = citations_histogram.get('refereed to nonrefereed', {}).get(relevant_year, 0)
                nonrefereed_to_refereed = citations_histogram.get('nonrefereed to refereed', {}).get(relevant_year, 0)
                nonrefereed_to_nonrefereed = citations_histogram.get('nonrefereed to nonrefereed', {}).get(relevant_year, 0)
                
                num_relevant_citations = (
                    refereed_to_refereed +
                    refereed_to_nonrefereed +
                    nonrefereed_to_refereed +
                    nonrefereed_to_nonrefereed
                )
                
                return [refereed_to_refereed, refereed_to_nonrefereed, nonrefereed_to_refereed, nonrefereed_to_nonrefereed, num_relevant_citations]
            else:
                print(f"No citation data found for bibcode {bibcode}")
                return [0, 0, 0, 0, 0]
        except json.JSONDecodeError as e:
            print(f"JSON decode error for bibcode {bibcode}: {e}")
            print(f"Response text: {response.text}")
            return -1
    else:
        print(f"Error {response.status_code} for bibcode {bibcode}: {response.text}")
        return -1    


In [34]:
journal_name = 'enter_journal_bibstem_here'
impact_factor_year = 2024

In [35]:
# Load the bibcodes from the json file
with open(f'{journal_name}/{journal_name}_bibcodes_{impact_factor_year}.json', 'r') as f:
    bibcodes = json.load(f)

In [36]:
len(bibcodes)

25

In [37]:
# Get the number of relevant citations for each bibcode
relevant_citations_list = []
start = 0
for bibcode in tqdm(bibcodes[start:]):
    num_relevant_citations = relevant_citations(impact_factor_year, bibcode)
    if num_relevant_citations == -1:
        break
    relevant_citations_list.append([bibcode, num_relevant_citations])
    
# Dump the relevant citations to a json file
with open(f'{journal_name}/temp/{journal_name}_relevant_citations_{impact_factor_year}_{start}.json', 'w') as f:
    json.dump(relevant_citations_list, f)
    
print("Done!")

100%|██████████| 25/25 [00:18<00:00,  1.34it/s]

Done!


In [38]:
# with open(f'{journal_name}/temp/{journal_name}_relevant_citations_{impact_factor_year}_{start}.json', 'w') as f:
#     json.dump(relevant_citations_list, f)
    
# print("Done!")

In [39]:
# relevant_citations_list[503]

In [40]:
bibcodes[0]

'2022ARA&A..60..121R'